In [1]:
from numpy import array
import pickle
import numpy as np
import random
from scipy import spatial
import itertools

from collections import ChainMap

from tqdm import tqdm


In [2]:
# Load the dictionary with only the item embeddings

item_embedddings = pickle.load( open( "item_embedddings.p", "rb" ) )
#print(len(item_embedddings))

151156


In [ ]:
## slicing the dictionary

sample_item_embedddings = dict(itertools.islice(item_embedddings.items(), 5))

In [3]:
### this cell stores the normalized item_embeddings
nomalized_item_embedddings = {}

for k,v in item_embedddings.items():
    v_norm = np.linalg.norm(v)
    nomalized_item_embedddings.update({k:v_norm})


In [4]:
# Load the output embeddings from LSTM

output_embeddings = pickle.load( open( "output_embeddings.p", "rb" ) )

In [5]:
## convert output embedings into 1 -D

new_output_embeddings = list(itertools.chain.from_iterable(output_embeddings)) 

sample_output_embeddings = new_output_embeddings[:3]


137998


In [6]:
from tqdm import tqdm

## this cell calculates the cosine similarity among y and y_hats
final_cosine = []
ranks = []


for yhat in tqdm(new_output_embeddings):  
    local_cosine = {}
    sorted_cosine = {}
    rank_list = []
    sliced_dict = {}
    yhat_norm = np.linalg.norm(yhat)
    original_item_embedding_values = list(item_embedddings.values())
    count = 0
    for k,v in nomalized_item_embedddings.items():
        
        # manually compute cosine similarity
        dot = np.dot(yhat, original_item_embedding_values[count])
        count +=1
        result = dot / (yhat_norm * v)
        local_cosine.update({k:result})
   
    sorted_cosine = {k: v for k, v in sorted(local_cosine.items(), key=lambda item: -item[1])}
    sliced_dict = dict(itertools.islice(sorted_cosine.items(), 50))  ## take top 50 prediction for each item
    
    rank_list = list(sliced_dict.keys())
    with open('ranks_index.p', 'ab') as f:
        pickle.dump(rank_list, f)
    del original_item_embedding_values


100%|██████████| 3/3 [00:04<00:00,  1.50s/it]


In [10]:
## this cell captures the top n ranked predicted items
with open('ranks_index.p', 'rb') as f: 
    y_hat = [pickle.load(f) for j in range(len(new_output_embeddings))] 

#y_hat

[['http://hm2.com/article#638526002187014',
  'http://hm2.com/article#695899002199005',
  'http://hm2.com/article#633051005187002',
  'http://hm2.com/article#636704001188005',
  'http://hm2.com/article#600229002188006'],
 ['http://hm2.com/article#638526002187014',
  'http://hm2.com/article#695899002199005',
  'http://hm2.com/article#600229002188006',
  'http://hm2.com/article#664647004188009',
  'http://hm2.com/article#633051005187002'],
 ['http://hm2.com/article#568047004187010',
  'http://hm2.com/article#676172003188013',
  'http://hm2.com/article#726199004199004',
  'http://hm2.com/article#732675001199003',
  'http://hm2.com/article#664387002188002']]

In [ ]:
## load original y labels

num_y_test = pickle.load( open( "original_output_embeddings.p", "rb" ) )

In [ ]:
## load original keys of y
y = []
original_output_embeddings = num_y_test.tolist()


for i in original_output_embeddings:
    for k,v in item_embedddings.items():
        if i ==v:
            y.append(k)

In [ ]:
## This cell is to generate the matrix for final learning
## so if original label is present in top n prediction labels , increase the count by 1
y_arr = [None]* len(y)
y_hat_arr = [None]* len(y_hat)
y_arr = y
y_hat_arr = y_hat

i = 0
count = 0
while(i<len(y_arr)):
    if (y_arr[i] in y_hat_arr[i]):
        count +=1
    i +=1
Ratio = count/len(y_arr)
print('Learning Ratio is : ',Ratio)